Cette unité est divisée en deux sections :
* Tout d'abord, nous verrons ce qui est nécessaire pour construire un système NLP capable de transformer un corps de texte en un tableau numérique de *caractéristiques*.
* Ensuite, nous montrerons comment réaliser ces étapes à l'aide d'outils réels.

# Construire un processeur de langage naturel à partir de zéro
Dans cette section, nous utiliserons les bases de Python pour construire un système de traitement du langage naturel rudimentaire. Nous construirons un *corpus de documents* (deux petits fichiers texte), créerons un *vocabulaire* à partir de tous les mots contenus dans les deux documents, puis nous démontrerons une technique de *bag of words* pour extraire des caractéristiques de chaque document.<br>
<div class="alert alert-info" style="margin: 20px">**Cette première section n'est donnée qu'à titre d'illustration !**
<br>Ne vous embêtez pas à mémoriser le code - nous ne ferions jamais cela dans la vie réelle.</div>

## Commencez par quelques documents :
Pour plus de simplicité, nous n'utiliserons pas de ponctuation.

In [22]:
%%writefile 1.txt
This is a story about cats
our feline pets
Cats are furry animals

Overwriting 1.txt


In [23]:
%%writefile 2.txt
This story is about surfing
Catching waves is fun
Surfing is a popular water sport

Overwriting 2.txt


## Construire un vocabulaire
L'objectif est ici de construire un tableau numérique à partir de tous les mots qui apparaissent dans chaque document. Plus tard, nous créerons des instances (vecteurs) pour chaque document individuel.

In [24]:
vocab = {}
i = 1

with open('1.txt') as f:
    x = f.read().lower().split()

for word in x:
    if word in vocab:
        continue
    else:
        vocab[word]=i
        i+=1

print(vocab)

{'this': 1, 'is': 2, 'a': 3, 'story': 4, 'about': 5, 'cats': 6, 'our': 7, 'feline': 8, 'pets': 9, 'are': 10, 'furry': 11, 'animals': 12}


In [25]:
with open('2.txt') as f:
    x = f.read().lower().split()

for word in x:
    if word in vocab:
        continue
    else:
        vocab[word]=i
        i+=1

print(vocab)

{'this': 1, 'is': 2, 'a': 3, 'story': 4, 'about': 5, 'cats': 6, 'our': 7, 'feline': 8, 'pets': 9, 'are': 10, 'furry': 11, 'animals': 12, 'surfing': 13, 'catching': 14, 'waves': 15, 'fun': 16, 'popular': 17, 'water': 18, 'sport': 19}


Même si `2.txt` contient 15 mots, seuls 7 nouveaux mots ont été ajoutés au dictionnaire.

## Extraction de caractéristiques
Maintenant que nous avons encapsulé notre "langue entière" dans un dictionnaire, effectuons une *extraction de caractéristiques* sur chacun de nos documents originaux :

In [26]:
# Create an empty vector with space for each word in the vocabulary:
one = ['1.txt']+[0]*len(vocab)
one

['1.txt', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [27]:
# map the frequencies of each word in 1.txt to our vector:
with open('1.txt') as f:
    x = f.read().lower().split()
    
for word in x:
    one[vocab[word]]+=1
    
one

['1.txt', 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]

<font color=green>Nous pouvons constater que la plupart des mots de 1.txt n'apparaissent qu'une seule fois, bien que "chats" apparaisse deux fois.</font>

In [7]:
# Do the same for the second document:
two = ['2.txt']+[0]*len(vocab)

with open('2.txt') as f:
    x = f.read().lower().split()
    
for word in x:
    two[vocab[word]]+=1

In [8]:
# Compare the two vectors:
print(f'{one}\n{two}')

['1.txt', 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
['2.txt', 1, 3, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1]


En comparant les vecteurs, nous constatons que certains mots sont communs aux deux, certains n'apparaissent que dans `1.txt`, d'autres que dans `2.txt`. En étendant cette logique à des dizaines de milliers de documents, nous verrions le dictionnaire de vocabulaire s'enrichir de centaines de milliers de mots. Les vecteurs contiendraient principalement des valeurs nulles, ce qui en ferait des *matrices éparses (sparse matrices)*.

## Bag of Words et Tf-idf
Dans les exemples ci-dessus, chaque vecteur peut être considéré comme un *sac de mots (bag of words)*. En soi, ceux-ci peuvent ne pas être utiles jusqu'à ce que nous considérions les *fréquences de termes (term frequencies)*, c'est-à-dire la fréquence d'apparition des mots individuels dans les documents. Une manière simple de calculer la fréquence des termes consiste à diviser le nombre d'occurrences d'un mot par le nombre total de mots dans le document. De cette manière, le nombre d'occurrences d'un mot dans les documents volumineux peut être comparé à celui des documents plus petits.

Cependant, il peut être difficile de différencier les documents sur la base de la fréquence des termes si un mot apparaît dans une majorité de documents. Dans ce cas, nous considérons également la *fréquence inverse du document (inverse document frequency)*, qui est le nombre total de documents divisé par le nombre de documents contenant le mot. En pratique, nous convertissons cette valeur sur une échelle logarithmique, comme suit [here](https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Inverse_document_frequency).


L'ensemble de ces termes devient [**tf-idf**](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

## Stop Words et Word Stems

Certains mots comme "the" et "and" apparaissent si souvent, et dans tant de documents, qu'il n'est pas nécessaire de les compter. De même, il peut être judicieux de n'enregistrer que la racine d'un mot, par exemple `cat` au lieu de `cat` et `cats`. Cela réduira notre tableau de vocabulaire et améliorera les performances.

## Tokenization et Tagging
Lorsque nous avons créé nos vecteurs, la première chose que nous avons faite a été de diviser le texte entrant sur les espaces blancs avec `.split()`. Il s'agissait d'une forme rudimentaire de *tokenisation*, c'est-à-dire de division d'un document en mots individuels. Dans cet exemple simple, nous ne nous sommes pas préoccupés de la ponctuation ou des différentes parties du discours. Dans le monde réel, nous nous appuyons sur une *morphologie* assez sophistiquée pour analyser le texte de manière appropriée.

Une fois le texte divisé, nous pouvons revenir en arrière et "étiqueter" nos tokens avec des informations sur les parties du discours, les dépendances grammaticales, etc. Cela ajoute des dimensions supplémentaires à nos données et permet une compréhension plus approfondie du contexte de documents spécifiques. C'est pour cette raison que les vecteurs deviennent des matrices éparses ***high dimensional sparse matrices***.

<div class="alert alert-info" style="margin: 20px">**C'est la fin de la première section.**
<br>Dans la section suivante, nous utiliserons scikit-learn pour effectuer une analyse réelle.</div>

___
# Extraction de caractéristiques à partir de textes
Dans le notebook**Scikit-learn Primer**, nous avons appliqué un modèle de classification SVC simple à l'ensemble de données SMSSpamCollection. Nous avons essayé de prédire l'étiquette ham/spam en nous basant sur la longueur du message et le nombre de ponctuations. Dans cette section, nous allons examiner le texte de chaque message et tenter d'effectuer une classification basée sur le contenu. Nous tirerons parti de certaines des fonctionnalités de scikit-learn [feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) tools.

## Charger le dataset

In [28]:
# Perform imports and load the dataset:
import numpy as np
import pandas as pd

df = pd.read_csv('./smsspamcollection.tsv', sep='\t')
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


## Vérifier les valeurs manquantes :
C'est toujours une bonne pratique.

In [29]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

## Jetez un coup d'œil rapide à la colonne `label` *ham* et *spam* :

In [30]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

<font color=green>4825 messages sur 5572, soit 86,6 %, sont des messages de jambon. Cela signifie que tout modèle de classification de texte que nous créons doit être **plus performant que 86,6 %** pour battre le hasard.</font>

## Diviser les données en ensembles d'entrainement et de test :

In [31]:
from sklearn.model_selection import train_test_split

X = df['message']  # this time we want to look at the text
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Scikit-learn's CountVectorizer

Le prétraitement du texte, la tokenisation et la possibilité de filtrer les stop words sont tous inclus dans [CountVectorizer] (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), qui construit un dictionnaire de caractéristiques et transforme les documents en vecteurs de caractéristiques (feature vectors).

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(3733, 7082)

<font color=green>Cela montre que notre ensemble d'entrainement est composé de 3733 documents et de 7082 caractéristiques.</font>

## Transformer les nombres en fréquences avec Tf-idf
Bien que le comptage des mots soit utile, les documents plus longs auront des valeurs moyennes de comptage plus élevées que les documents plus courts, même s'ils traitent des mêmes sujets.

Pour éviter cela, nous pouvons simplement diviser le nombre d'occurrences de chaque mot dans un document par le nombre total de mots dans le document : ces nouvelles caractéristiques sont appelées **tf** pour Term Frequencies.

Un autre raffinement de **tf** consiste à réduire les poids des mots qui apparaissent dans de nombreux documents du corpus et qui sont donc moins informatifs que ceux qui n'apparaissent que dans une plus petite partie du corpus.

Cette réduction d'échelle est appelée **tf-idf** pour "Term Frequency times Inverse Document Frequency" (fréquence des termes multipliée par la fréquence inverse des documents).

Le tf et le tf-idf peuvent être calculés comme suit en utilisant [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html):

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(3733, 7082)

Note : la méthode `fit_transform()` effectue en fait deux opérations : elle ajuste un estimateur aux données et transforme ensuite notre matrice de comptage en une représentation tf-idf.

## Combiner les étapes avec TfidVectorizer
À l'avenir, nous pourrons combiner les étapes CountVectorizer et TfidTransformer en une seule à l'aide de la fonction [TfidVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html):

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(3733, 7082)

## Former un classificateur
Nous présentons ici un classificateur SVM similaire au SVC, appelé [LinearSVC] (https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html). LinearSVC gère mieux les données éparses et s'adapte bien à un grand nombre d'échantillons.

In [35]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

<font color=green>Plus tôt, nous avons nommé notre classifieur SVC **svc_model**. Ici, nous utilisons le nom plus générique **clf** (pour classifier).</font>

## Construire un pipeline
Rappelez-vous que seul notre ensemble d'apprentissage a été vectorisé en un vocabulaire complet. Afin d'effectuer une analyse sur notre jeu de test, nous devrons le soumettre aux mêmes procédures. Heureusement, scikit-learn propose une classe [**Pipeline**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) qui se comporte comme un classificateur composé.

In [36]:
from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

C:\Users\aluboya\.conda\envs\nlp_course\lib\site-packages\sklearn\utils\validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

## Tester le classifieur et afficher les résultats

In [37]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [38]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [39]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

   micro avg       0.99      0.99      0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [40]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.989668297988037


En utilisant le texte des messages, notre modèle a obtenu d'excellents résultats : il a prédit correctement le spam dans 98,97 % des cas !
Appliquons maintenant ce que nous avons appris à un projet de classification de texte impliquant des critiques de films positives et négatives.

## Suivant : Projet de classification de texte